## Create and compile the model - Centralised approach - deadline 26 June

The model will be later used for training on each client.

Target 1: Train a centralised model and calculate performance.

Target 2: Test various models (imagenet etc.)

Target 3: Write documentation about centralised model.

In [1]:
import torch
from torchvision import datasets, transforms    # import datasets and transforms from torchvision
import matplotlib.pyplot as plt    # import matplotlib.pyplot to plot images
import numpy as np  # import numpy to perform numerical operations
from tqdm.notebook import tqdm  # import tqdm to display progress bar

# Set the device for google colaboratory
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

device = "cuda"

#print(f"Using device: {device}")

#device = "mps" if torch.backends.mps.is_available() else "cpu"
#print(f"Using device: {device}")


#################################### Data Loading ##########################################################
############################################################################################################

batch_size = 100 # Define batch size

# Define a transform to normalize the data
# Use compose to chain multiple transforms together
transform = transforms.Compose([transforms.ToTensor(),  # Convert the image to PyTorch tensor
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), # Normalize the data with mean and standard deviation
                                ])

# Download the training data and test data
trainset = datasets.CIFAR10(root='./data', download=True, transform=transform) # CIFAR10 is a dataset of natural images belonging to 10 different classes
testset = datasets.CIFAR10(root='./data', download=True, train=False, transform=transform) #    train=False means we want the test data and not the training data

# Split the training data into training and validation sets
trainset, valset = torch.utils.data.random_split(trainset, [40000, 10000]) # 40000 is the number of images in the training set and 10000 is the number of images in the validation set

# Print the number of images in the training and test sets
print("Number of training images: ", len(trainset))
print("Number of validation images: ", len(valset))
print("Number of test images: ", len(testset))

# Create a dataloader to load the data in batches
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)    # shuffle=True means the data will be shuffled at every epoch
                                                                                    # batch_size=64 means we will load 64 images at a time
# Create a dataloader to load the validation data in batches
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False)     # We use a dataloader to load the data in batches and improve validation performance
                                                                                    # shuffle=False means the data will not be shuffled since we want to validate on the validation data
                                                                                    # batch_size=64 means we will load 64 images at a time

# Create a dataloader to load the test data in batches
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)     # We use a dataloader to load the data in batches and improve test performance
                                                                                    # shuffle=False means the data will not be shuffled since we want to test on the test data
                                                                                    # batch_size=64 means we will load 64 images at a time

Wed Nov 22 11:49:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

100%|██████████| 170498071/170498071 [00:13<00:00, 12840362.83it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Number of training images:  40000
Number of validation images:  10000
Number of test images:  10000


In [ ]:
#################################### Model Structure #######################################################
############################################################################################################

# Load VGGNet16 model from torchvision and modify it to accept 32x32 images and 3 channels
# model = torch.hub.load('pytorch/vision:v0.6.0', 'vgg16', pretrained=True) # pretrained=True means we want to use the pretrained weights of the model
# model.features[0] = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1) # 3 is the number of input channels and 64 is the number of output channels
# model.classifier[6] = torch.nn.Linear(4096, 10) # 4096 is the number of input features and 10 is the number of output features

# Load the resnet18 model
resnet18_model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True) # pretrained=True means we want to use the pretrained weights of the model

# Load the resnet34 model
resnet34_model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet34', pretrained=True) # pretrained=True means we want to use the pretrained weights of the model

# Create a function that modifies resnet18 and resnet34 so that they can be applied to CIFAR10 dataset
def modify_resnet(model):
    model.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False) # 3 is the number of input channels and 64 is the number of output channels
    model.maxpool = torch.nn.Identity() # Remove the maxpool layer
    model.avgpool = torch.nn.AdaptiveAvgPool2d(1) # Add an adaptive average pooling layer
    model.fc = torch.nn.Linear(512, 10) # 512 is the number of input features and 10 is the number of output features
    return model

# Visualise the modified resnet18 model
resnet18_model = modify_resnet(resnet18_model)
print(resnet18_model)



# Modify the last layer of the model to have 10 outputs instead of 1000
#model.fc = torch.nn.Linear(512, 10) # 512 is the number of input features and 10 is the number of output features
# Modify the first layer of the model to accept 32x32 images instead of 224x224 images and 3 channels instead of 1 channel
#model.conv1 = torch.nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False) # 3 is the number of input channels and 64 is the number of output channels
############################################################################################################
############################################################################################################import torch

Using cache found in /Users/scmts1/.cache/torch/hub/pytorch_vision_v0.6.0
/Users/scmts1/opt/anaconda3/envs/tensorflow/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/scmts1/opt/anaconda3/envs/tensorflow/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Using cache found in /Users/scmts1/.cache/torch/hub/pytorch_vision_v0.6.0
/Users/scmts1/opt/anaconda3/envs/tensorflow/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` f

  0%|          | 0.00/83.3M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), p

In [ ]:

#################################### Training ##############################################################
############################################################################################################

model.to(device)

# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.7)

# Initialize lists to store the losses and accuracies
train_losses = []
val_losses = []
val_accuracies = []

# Train the model
for epoch in range(4):
    running_loss = 0.0
    for i, data in enumerate(tqdm(trainloader, desc=f'Epoch {epoch+1}'), 0):
        inputs, labels = data

        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate validation loss and accuracy every 50 batches
        if i % 50 == 49:
            val_loss = 0.0
            correct = 0
            total = 0
            with torch.no_grad():
                for data in valloader:
                    images, labels = data

                    images = images.to(device)
                    labels = labels.to(device)

                    outputs = model(images)
                    val_loss = criterion(outputs, labels)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                    val_losses.append(val_loss)
                    val_accuracies.append(100 * correct / total)

            avg_loss = running_loss / 50
            train_losses.append(avg_loss)
            running_loss = 0.0

    print(f'Epoch {epoch+1} training loss: {avg_loss:.3f} validation accuracy: {100 * correct / total:.2f}%')

print('Finished Training')
torch.save(model.state_dict(), './VGGNet16.pth')

In [ ]:

# Create a figure for plotting the training loss
plt.figure()
plt.title('Training Loss')
plt.xlabel('100Minibatches')
plt.ylabel('Loss')
plt.plot(np.arange(len(losses)), losses) # Plot the losses vs the number of mini-batches

val_losses = [val.cpu() for val in val_losses]
# Create a figure for plotting the validation loss
plt.figure()
plt.title('Validation Loss')
plt.xlabel('100Minibatches')
plt.ylabel('Loss')
plt.plot(np.arange(len(val_losses)), val_losses) # Plot the validation losses vs the number of mini-batches

# Create a figure for plotting the validation accuracy
plt.figure()
plt.title('Validation Accuracy')
plt.xlabel('100Minibatches')
plt.ylabel('Accuracy')
plt.plot(np.arange(len(val_accuracies)), val_accuracies) # Plot the validation accuracies vs the number of mini-batches

# test the model on the test data
correct = 0
total = 0
with torch.no_grad():   # We don't need to calculate gradients since we are not training the model
    for data in testloader: # Iterate over the test data
        images, labels = data[0].to(device), data[1].to(device) # Get the inputs and labels from the data and move them to the device
        outputs = model(images) # Get the outputs from the model
        _, predicted = torch.max(outputs.data, 1) # Get the predicted labels
        total += labels.size(0) # Increment the total number of images
        correct += (predicted == labels).sum().item() # Increment the number of correctly predicted images

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total)) # Print the accuracy of the model on the test data

To use k-fold cross validation in your code, you can modify the training loop to use k-fold cross validation instead of training on the entire training set. Here's an example of how you can modify the code to use k-fold cross validation:

```python
import torch
from torchvision import datasets, transforms
from sklearn.model_selection import KFold
import numpy as np

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                ])

# Download the data
trainset = datasets.CIFAR10(root='./data', download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', download=True, train=False, transform=transform)

# Define the number of folds
k = 5

# Create a KFold object to split the data into k folds
kf = KFold(n_splits=k, shuffle=True)

# Define the model
model = torch.hub.load('pytorch/vision:v0.6.0', 'vgg16', pretrained=True)
model.features[0] = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
model.classifier[6] = torch.nn.Linear(4096, 10)
model.to(device)

# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Define a list to store the accuracies for each fold
accuracies = []

# Loop over the folds
for fold, (train_indices, val_indices) in enumerate(kf.split(trainset)):
    print(f"Fold {fold+1}/{k}")
    


In [ ]:
# Create a dataloader for the training data for this fold
trainloader = torch.utils.data.DataLoader(torch.utils.data.Subset(trainset, train_indices), batch_size=64, shuffle=True)

# Create a dataloader for the validation data for this fold
valloader = torch.utils.data.DataLoader(torch.utils.data.Subset(trainset, val_indices), batch_size=64, shuffle=False)

# Train the model for this fold
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}: Training loss = {running_loss/len(trainloader)}")

    # Evaluate the model on the validation data for this fold
    correct = 0
    total = 0
    with torch.no_grad():
        for data in valloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}: Validation accuracy = {accuracy}%")

#

In [ ]:
#print F1 score and confusion matrix
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import torch
import numpy as np
import seaborn as sn
from torchvision import datasets, transforms    # import datasets and transforms from torchvision


device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

# set model from saved model
model = torch.hub.load('pytorch/vision:v0.6.0', 'vgg16', pretrained=True)
model.features[0] = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
model.classifier[6] = torch.nn.Linear(4096, 10)
model.load_state_dict(torch.load('./VGGNet16.pth'))
model.to(device) # Move the model to the device

# Define a transform to normalize the data
# Use compose to chain multiple transforms together
transform = transforms.Compose([transforms.ToTensor(),  # Convert the image to PyTorch tensor
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), # Normalize the data with mean and standard deviation
                                ])

# Download the training data and test data
testset = datasets.CIFAR10(root='./data', download=True, train=False, transform=transform) #    train=False means we want the test data and not the training data


print("Number of test images: ", len(testset))
                                                                                   # batch_size=64 means we will load 64 images at a time
# Create a dataloader to load the test data in batches
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)     # We use a dataloader to load the data in batches and improve test performance




# test the model on the test data
correct = 0
total = 0
y_true = []
y_pred = []

with torch.no_grad():   # We don't need to calculate gradients since we are not training the model
    for data in testloader: # Iterate over the test data
        images, labels = data[0].to(device), data[1].to(device) # Get the inputs and labels from the data and move them to the device
        outputs = model(images) # Get the outputs from the model
        _, predicted = torch.max(outputs.data, 1) # Get the predicted labels
        total += labels.size(0) # Increment the total number of images
        correct += (predicted == labels).sum().item() # Increment the number of correctly predicted images
        y_true += labels.tolist()
        y_pred += predicted.tolist()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total)) # Print the accuracy of the model on the test data

# Print the F1 score
print('F1 score: ', f1_score(y_true, y_pred, average='macro'))

# visualise the confusion matrix as a heatmap
cm = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(cm, range(10), range(10))
plt.figure(figsize=(10,7))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 12}) # font size
plt.show()



## Implement model attacks

Implement a function to simulate a model attack and measure the effect of the attack against existing methods

Target 1: Perform literature review on simulating model attacks (ART seems to be the best option, but this is to simulate the actual attack; perhaps most papers use other simpler ways to implement an attack)

Target 2: Implement model attack simulation based on literature review performed in target 1

Target 3: Measure model performance (using the model that was trained in the previous stage) before and after the attack

## Implement Federated Learning architecture

Implement FL relying on the existing models from stage 1

Target 1: Perform literature review on existing FL frameworks with a focus on those used for cybersecurity experimentation on model attacks

Target 2: Implement FL relying on the model from stage 1

Target 3: Train and evaluate the results on each client

## Implement Federated Learning attack

Target 1: Perform literature review on model attacks in Federated Learning (you already have the model attack form earlier stage, but there might be better ways to attack in order to make the attack persistent for example...)

Target 2: Implement the attack(s) and measure the performance of the model (on each client and the Global model) in time

## Implement Defence and measure effectiveness

Target 1: Implement the proposed defence

Target 2: Measure performance on each client and centrally for each type of attack in the previous step after applying the proposed security mechanism.

## Compare against other defence methods